In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
from datetime import datetime
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import EarthLocation
from astropy.coordinates import builtin_frames
from astropy.coordinates import Angle
from bokeh.io import output_notebook, output_file
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Whisker, Step, Title, LinearColorMapper, BasicTicker, ColorBar
from bokeh.layouts import row
from bokeh.palettes import Inferno256, Greys9

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
data = [0]*(199+201)
for i in range(0,199):
    with h5py.File('/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/20201118/dl2_LST-1.Run02929.{:04d}.h5'
               .format(i), 'r') as f:
        data[i] = pd.DataFrame(np.array(f['dl2/event/telescope/parameters/LST_LSTCam']))
        
for i in range(0,201):
    with h5py.File('/nfs/cta-ifae/moralejo/CTA/LST/RealData/DL2/20201118/dl2_LST-1.Run02930.{:04d}.h5'
               .format(i), 'r') as f:
         data[199+i] = pd.DataFrame(np.array(f['dl2/event/telescope/parameters/LST_LSTCam']))
            
tot_data = pd.concat([data[i] for i in range(0,199+201)])
tot_data.head()

,intensity,log_intensity,x,y,r,phi,length,width,psi,skewness,...,log_reco_energy,reco_energy,reco_disp_dx,reco_disp_dy,reco_src_x,reco_src_y,reco_alt,reco_az,reco_type,gammaness
0,14019.949485,4.146746,-0.015204,-0.005036,0.016016,-2.821747,1.173447,1.142609,0.518989,0.023384,...,1.161159,14.493039,0.586236,0.238408,0.571033,0.233373,1.281348,1.902129,101,0.081857
1,76.737119,1.885005,-0.527041,-0.305895,0.609380,-2.615709,0.079877,0.073439,0.426368,0.060108,...,-1.459769,0.034692,-0.153171,-0.079658,-0.680212,-0.385553,1.236501,1.830964,101,0.386667
2,25.876774,1.412910,-0.349313,0.001244,0.349315,3.138032,0.088380,0.040477,0.769460,0.026443,...,-1.569809,0.026927,0.050654,0.022915,-0.298659,0.024159,1.250402,1.875670,101,0.305238
3,25.759664,1.410940,0.122801,-0.445033,0.461665,-1.301559,0.061317,0.053258,-1.157097,0.232783,...,-1.334617,0.046279,0.056081,-0.049218,0.178882,-0.494252,1.266962,1.813900,101,0.226262
4,44.028748,1.643736,-0.141567,0.933637,0.944309,1.721280,0.080193,0.045229,-1.103720,0.250541,...,-1.414461,0.038507,0.080160,0.050695,-0.061408,0.984333,1.256965,1.987032,101,0.251333


In [4]:
tot_data.shape

(18213591, 50)

In [5]:
tot_data.columns

Index(['intensity', 'log_intensity', 'x', 'y', 'r', 'phi', 'length', 'width',
       'psi', 'skewness', 'kurtosis', 'time_gradient', 'intercept',
       'leakage_intensity_width_1', 'leakage_intensity_width_2',
       'leakage_pixels_width_1', 'leakage_pixels_width_2', 'n_pixels',
       'concentration_cog', 'concentration_core', 'concentration_pixel',
       'n_islands', 'alt_tel', 'az_tel', 'obs_id', 'event_id',
       'calibration_id', 'dragon_time', 'ucts_time', 'tib_time', 'mc_type',
       'mc_core_distance', 'wl', 'tel_id', 'tel_pos_x', 'tel_pos_y',
       'tel_pos_z', 'trigger_type', 'ucts_trigger_type', 'trigger_time',
       'log_reco_energy', 'reco_energy', 'reco_disp_dx', 'reco_disp_dy',
       'reco_src_x', 'reco_src_y', 'reco_alt', 'reco_az', 'reco_type',
       'gammaness'],
      dtype='object')

## Calculate the equatorial coordinates (right ascension and declination) of the events + make a skymap of the reconstructed directions of the events in these coordinates.

In [6]:
def utc(df):   #function that calculates the utc time of the events  
    utc_time = []
    for i in range(df.shape[0]):
        idx = list(df.columns).index('dragon_time')
        a = datetime.utcfromtimestamp(df.iloc[i,idx])   #idx corresponds to the index of the column 'dragon_time'
        utc_time.append(a)
        
    return utc_time

In [7]:
def equatorial_coords(df):   #function that calculates the equatorial coordinates (ra and dec) of the events from the horizontal coordinates 
    utc_time = []
    for i in range(df.shape[0]):
        idx = list(df.columns).index('dragon_time')
        a = datetime.utcfromtimestamp(df.iloc[i,idx])   #idx corresponds to the index of the column 'dragon_time'
        utc_time.append(a)

    df = df.assign(utc_time = utc_time)   #add new column to the data frame with the UTC observation times of the events

    loc = EarthLocation(lat = 28.76152611*u.deg, lon = -17.89149701*u.deg, height = 2184*u.m)   #location of the telescope (CTA LST1: Roque de los Muchachos)

    hor_coords = SkyCoord(alt = df['reco_alt'], az = df['reco_az'], frame = 'altaz', unit = 'rad', 
                          obstime = df['utc_time'], location = loc)   #horizontal coordinates of the events
    eq_coords = hor_coords.icrs   #equatorial coordinates of the events
    
    return eq_coords

In [16]:
data = tot_data[(tot_data['gammaness']>0.8)]

data = data.assign(utc_time = utc(data))   #add new column to the data frame with the UTC observation times of the events
data = data.assign(RA = equatorial_coords(data).ra)   #add columns with right ascension and declination (equatorial coordinates) of the events
data = data.assign(DEC = equatorial_coords(data).dec)

# skymap (RA, dec) de las posiciones reconstruidas de los sucesos (ya se ha aplicado el corte de gammaness > 0.8):
counts, bins_x, bins_y = np.histogram2d(data['RA'], data['DEC'], bins=250)
fig = figure(x_range=(min(bins_x), max(bins_x)), y_range=(min(bins_y), max(bins_y)), plot_width=500, plot_height=300)
fig.image(image=[counts], x=bins_x[0], y=bins_y[0], dw=bins_x[-1] - bins_x[0], dh=bins_y[-1] - bins_y[0], palette=Inferno256) 
fig.xaxis.axis_label = 'right ascension (deg)'
fig.yaxis.axis_label = 'declination (deg)'
color_mapper = LinearColorMapper(palette=Inferno256, low=counts.min(), high=counts.max())
color_bar = ColorBar(color_mapper=color_mapper, ticker= BasicTicker(), location=(0,0))
fig.add_layout(color_bar, 'right')
show(fig)

## Ver que hay un exceso significativo de sucesos (gammas) en las coordenadas del Crab y estimar el número de gammas (haciendo "aperture photometry"), tomando 3 zonas off creando una cruz con la posición del Crab (para tener más estadística para calcular el fondo).

In [17]:
def off_zones(df):   #function that defines the three mentioned off zones and returns the angular separation between the position of each event and each off zone
    crab = SkyCoord.from_name('M1')    #equatorial coordinates of the Crab nebula in deg

    eq_coords = SkyCoord(df['RA'], df['DEC'], frame='icrs', unit='deg')   #frame 'icrs' = equatorial coordinates (of the events)
    theta2 = (eq_coords.separation(crab))**2  #square of the angular separation between the events and the crab (in deg)

    loc = EarthLocation(lat = 28.76152611*u.deg, lon = -17.89149701*u.deg, height = 2184*u.m)
    hor_coords_tel = SkyCoord(alt = df['alt_tel'], az = df['az_tel'], frame = 'altaz', unit = 'rad', 
                          obstime = df['utc_time'], location = loc)   #horizontal coordinates of the telescope
    eq_coords_tel = hor_coords_tel.icrs   #equatorial coordinates of the telescope

    skyoffset_frame = builtin_frames.SkyOffsetFrame(origin = eq_coords_tel)  #define a new reference frame centered at the pointing direction of the telescope at each moment

    crab2 = crab.transform_to(skyoffset_frame)   #position of the Crab nebula with respect to the new frame 

    pos = [0]*3
    theta2_off = [0]*3
    pos[0] = SkyCoord(-crab2.lon, -crab2.lat, frame = skyoffset_frame, unit = 'deg')  #symmetric position to the position of the Crab with respect to the center of the FOV
    pos[1] = SkyCoord(Angle(crab2.lat), Angle(-crab2.lon), frame = skyoffset_frame, unit = 'deg') 
    pos[2] = SkyCoord(Angle(-crab2.lat), Angle(crab2.lon), frame = skyoffset_frame, unit = 'deg') 

    for i in range(3):
        pos[i] = pos[i].transform_to('icrs') 
        theta2_off[i] = (eq_coords.separation(pos[i]))**2   #square of the angular separation between the events and each of the off zones
        
    return theta2, theta2_off

In [18]:
def gamma_excess(df):    #function that returns the excess of gamma rays of a given dataframe (with a given cut in gammaness
    par = ['theta2', 'theta2_off1', 'theta2_off2', 'theta2_off3']

    n = [0]*4
    b = [0]*4
    colors = ['blue', 'orange', 'green', 'magenta']
    fig1 = figure(plot_width=600, plot_height=400, x_range=(0,2.5))
    for i in range(4):
        n[i], b[i] = np.histogram(df[par[i]], bins=750)
        
        source = ColumnDataSource(dict(x=(b[i][1:]+b[i][:-1])/2, y=n[i]))
        glyph1 = Step(x='x', y='y', line_color=colors[i], mode='center')
        fig1.add_glyph(source, glyph1)

        base = (b[i][1:]+b[i][:-1])/2
        lower = n[i]-np.sqrt(n[i])/2
        upper = n[i]+np.sqrt(n[i])/2
        source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))
        w=Whisker(source=source_error, base='base', upper='upper', lower='lower', line_color='gray')
        w.upper_head.line_color = 'gray'
        w.lower_head.line_color = 'gray'
        fig1.add_layout(w)
    
    fig1.xaxis.axis_label = 'theta²'
    fig1.yaxis.axis_label = 'counts'
    show(fig1)

    excess_gammas = n[0][0]+n[0][1]-np.mean([n[1][0],n[2][0],n[3][0]])-np.mean([n[1][1],n[2][1],n[3][1]])
    return print('Excess of gamma rays: {}'. format(excess_gammas))

## Obtain the distribution of theta², width, length and intensity of (only) the gamma rays.

#### *** ERROR PROPAGATION: 

($n_1$, $n_2$, $n_3$, $n_4$): histogram bins (for each position) 

($\sqrt{n_1}$, $\sqrt{n_2}$, $\sqrt{n_3}$, $\sqrt{n_4}$): errors of each "variable" ($n_i$)

new variable (function of $(n_1, n_2, n_3, n_4)$): $n(n_1, n_2, n_3, n_4)=n_1-\frac{1}{3}(n_2+n_3+n_4)$

Covariance matrix of the variables $n_i$ (since $\sigma_i=\sqrt{n_i}$ and $cov(x_i,x_i)=\sigma_i²$): 

\begin{equation*}
V = cov[n_i,n_j]=
\begin{pmatrix}
n_1 &  &  & \\
& n_2 &  & \\
 &  & n_3 &  \\
 &  &  &  n_4
\end{pmatrix}
\end{equation*}

To obtain the variance $U$ of the variable $n$, we use error propagation: 
$$U = \sum_{k,l=1}^n \frac{\partial n}{\partial n_i} \frac{\partial n}{\partial n_j} V_{kl}$$
$$A_1 = \frac{\partial n}{\partial n_1}=1  ; \quad A_2=A_3=A_4=-1/3 \quad \Rightarrow U=V_{11}+\frac{1}{9}(V_{22}+V_{33}+V_{44})$$

In [19]:
def dfs(df):   
    gammas = df[(df['theta2']<0.05)]   #dataframe that contains only the events 0.05 degrees around the position of the crab (gammas + bkg of cosmic rays)
    bkg2 = df[(df['theta2_off1']<0.05)]   #dataframe that contains only the events 0.05 degrees around the off zone 1 (bkg of cosmic rays)
    bkg3 = df[(df['theta2_off2']<0.05)]   #dataframe that contains only the events 0.05 degrees around the off zone 2 (bkg of cosmic rays)
    bkg4 = df[(df['theta2_off3']<0.05)]   #dataframe that contains only the events 0.05 degrees around the off zone 3 (bkg of cosmic rays)
    
    dfs = [gammas, bkg2, bkg3, bkg4]
    return dfs

In [20]:
def distribution(df, parameter, bins, xlims = None):   #function that returns the distribution of a given parameter using a given dataframe (with a given cut in gammaness)  
    idx = list(df.columns).index(parameter)
    par = df.columns[idx] 
    
    n = [0]*4
    b = [0]*4
    colors = ['blue', 'orange', 'green', 'magenta']
    fig1 = figure(title = 'Distribution of {} of the gamma rays + background of cosmic rays'.format(par),
                  plot_width=450, plot_height=300, x_range=xlims)
    for i in range(4):
        name = dfs(df)[i]
        n[i], b[i] = np.histogram(name[par], bins=bins, range = (min(df[par]),max(df[par])))
        
        source = ColumnDataSource(dict(x=(b[i][1:]+b[i][:-1])/2, y=n[i]))
        glyph1 = Step(x='x', y='y', line_color=colors[i], mode='center')
        fig1.add_glyph(source, glyph1)

        base = (b[i][1:]+b[i][:-1])/2
        lower = n[i]-np.sqrt(n[i])/2
        upper = n[i]+np.sqrt(n[i])/2
        source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))
        w=Whisker(source=source_error, base='base', upper='upper', lower='lower', line_color='gray')
        w.upper_head.line_color = 'gray'
        w.lower_head.line_color = 'gray'
        fig1.add_layout(w)
    
    fig1.xaxis.axis_label = parameter
    fig1.yaxis.axis_label = 'counts'
    fig1.title.text_font_size = '8pt'
    fig1.title.text_font_style = 'normal'
    
    N = n[0]-(1/3)*(n[1]+n[2]+n[3])  #histogram bins of only the gamma rays (without the bkg of cosmic rays)
    err_N = np.sqrt(n[0]+1/9*(n[1]+n[2]+n[3]))  #error of N, obtained by error propagation
    width = b[1][1] - b[1][0]
    
    fig2 = figure(title = 'Distribution of {} of the gamma rays'.format(par),
                  plot_width=450, plot_height=300, x_range=xlims)
    fig2.quad(top=N, bottom=0, left=b[1][:-1], right=b[1][1:], width=width)
    base = (b[1][1:]+b[1][:-1])/2
    lower = N - err_N
    upper = N + err_N
    source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))
    w=Whisker(source=source_error, base='base', upper='upper', lower='lower', line_color='gray')
    w.upper_head.line_color = 'gray'
    w.lower_head.line_color = 'gray'
    fig2.add_layout(w)

    fig2.xaxis.axis_label = parameter
    fig2.yaxis.axis_label = 'counts'
    fig2.title.text_font_size = '8pt'
    fig2.title.text_font_style = 'normal'
    fig2.add_layout(Title(text='(with the background of cosmic rays substracted)', text_font_style='normal', text_font_size='8pt'), 'above')
    
    show(row(fig1, fig2))

In [21]:
def hist_intensity(df, parameter, bins):  #function that returns the plot of a given parameter in bins of intensity
    idx = list(df.columns).index(parameter)
    par = df.columns[idx] 
    
    counts = []
    for i in range(4):
        name = dfs(df)[i]
        (counts1, bins_x1, bins_y1) = np.histogram2d(name[par], name['intensity'], bins = bins)
        counts.append(counts1)
        
    counts_gamma = counts[0] - (1/3)*(counts[1] + counts[2] + counts[3])
    
    fig = figure(title='Intensity of the gamma ray events as a function of the {}'.format(parameter), plot_width=450, plot_height=400, x_range=(min(bins_x1), max(bins_x1)), y_range=(min(bins_y1), max(bins_y1)))
    fig.image(image=[np.transpose(counts_gamma)], x=bins_x1[0], y=bins_y1[0], dw=bins_x1[-1] - bins_x1[0], 
              dh=bins_y1[-1] - bins_y1[0])
    fig.xaxis.axis_label = parameter
    fig.yaxis.axis_label = 'intensity'
    fig.title.text_font_size = '8pt'
    fig.title.text_font_style = 'normal'
    
    color_mapper = LinearColorMapper(palette=Greys9, low=counts_gamma.min(), high=counts_gamma.max())
    color_bar = ColorBar(color_mapper=color_mapper, ticker= BasicTicker(), location=(0,0))
    fig.add_layout(color_bar, 'right')
    
    show(fig)

# We apply these functions to the data with a cut in gammaness >0.8. 

In [22]:
data1 = tot_data[(tot_data['gammaness']>0.8) & (tot_data['intensity']>200)]

In [23]:
data1 = data1.assign(utc_time = utc(data1))   #add new column to the data frame with the UTC observation times of the events
data1 = data1.assign(RA = equatorial_coords(data1).ra)   #add columns with right ascension and declination (equatorial coordinates) of the events
data1 = data1.assign(DEC = equatorial_coords(data1).dec)
data1 = data1.assign(theta2 = off_zones(data1)[0])
data1 = data1.assign(theta2_off1 = off_zones(data1)[1][0])
data1 = data1.assign(theta2_off2 = off_zones(data1)[1][1])
data1 = data1.assign(theta2_off3 = off_zones(data1)[1][2])

In [24]:
gamma_excess(data1)

Excess of gamma rays: 209.66666666666666


In [25]:
distribution(data1, 'theta2', 400, xlims=(0,1.5))

In [48]:
distribution(data1, 'width', 50)

In [49]:
distribution(data1, 'length', 40)

In [51]:
distribution(data1, 'log_intensity', 30)

In [29]:
hist_intensity(data1, 'width', 100)

In [30]:
hist_intensity(data1, 'length', 100)

## Do the same (obtain the distributions of some parameters) but with a less hard cut in gammaness: gammaness > 0.5

In [31]:
data2 = tot_data[(tot_data['gammaness']>0.5) & (tot_data['intensity']>200)]

In [32]:
data2 = data2.assign(utc_time = utc(data2))   #add new column to the data frame with the UTC observation times of the events
data2 = data2.assign(RA = equatorial_coords(data2).ra)   #add columns with right ascension and declination (equatorial coordinates) of the events
data2 = data2.assign(DEC = equatorial_coords(data2).dec)
data2 = data2.assign(theta2 = off_zones(data2)[0])
data2 = data2.assign(theta2_off1 = off_zones(data2)[1][0])
data2 = data2.assign(theta2_off2 = off_zones(data2)[1][1])
data2 = data2.assign(theta2_off3 = off_zones(data2)[1][2])

In [33]:
gamma_excess(data2)

Excess of gamma rays: 1005.3333333333335


In [34]:
distribution(data2, 'theta2', 400, xlims = (0,1.5))

In [35]:
distribution(data2, 'width', 50)

In [36]:
distribution(data2, 'length', 50)

In [37]:
distribution(data2, 'log_intensity', 40)

In [38]:
hist_intensity(data2, 'width', 150)

In [39]:
hist_intensity(data2, 'length', 150)